In [1]:
import ray

In [2]:
ray.shutdown()

In [3]:
ray.init()

2021-06-04 10:55:36,533	INFO services.py:1269 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.37',
 'raylet_ip_address': '192.168.1.37',
 'redis_address': '192.168.1.37:6379',
 'object_store_address': '/tmp/ray/session_2021-06-04_10-55-35_103935_66596/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-06-04_10-55-35_103935_66596/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-06-04_10-55-35_103935_66596',
 'metrics_export_port': 62153,
 'node_id': 'b55f2340f68d5439d9728c6605bd1ae90b31d893b63c92dea02643fd'}

In [4]:
%config IPCompleter.use_jedi = False

In [5]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MinMaxScaler

transformer = FunctionTransformer(np.log1p)
minmax_scaler = MinMaxScaler()

In [7]:
import codeflare.pipelines.Datamodel as dm

class FeatureUnion(dm.AndEstimator):
    def __init__(self):
        pass
    
    def fit_transform(self, xy_list: list):
        return self.transform(xy_list)

    def get_estimator_type(self):
        return 'transform'
    
    def transform(self, xy_list):
        X_list = []
        y_list = []
        
        for xy in xy_list:
            X_list.append(xy.get_x())
        X_concat = np.concatenate(X_list, axis=0)
        
        return dm.Xy(X_concat, None)

In [8]:
pipeline = dm.Pipeline()

node_a = dm.EstimatorNode('a', minmax_scaler)
node_b = dm.EstimatorNode('b', minmax_scaler)
node_c = dm.AndNode('c', FeatureUnion())
# node_d = dm.OrNode('d', transformer)

pipeline.add_edge(node_a, node_c)
pipeline.add_edge(node_b, node_c)
# pipeline.add_edge(node_b, node_d)

In [9]:
X = np.array([0, 1, 2, 3])
X = np.reshape(X, (4, 1))

In [10]:
import codeflare.pipelines.Datamodel as dm

In [11]:
pipeline_input = dm.PipelineInput()
xy = dm.Xy(X, None)
pipeline_input.add_xy_arg(node_a, xy)
pipeline_input.add_xy_arg(node_b, xy)

In [12]:
import codeflare.pipelines.Runtime as rt

In [13]:
from codeflare.pipelines.Runtime import ExecutionType

In [14]:
pipeline_output = rt.execute_pipeline(pipeline, ExecutionType.FIT, pipeline_input)

In [15]:
out_Xyrefs = pipeline_output.get_xyrefs(node_c)

In [17]:
for out_xyref in out_Xyrefs:
    x = ray.get(out_xyref.get_Xref())
    and_func = ray.get(out_xyref.get_curr_node_state_ref()).get_estimator()
    print(x)
    print(and_func)

[[0.        ]
 [0.33333333]
 [0.66666667]
 [1.        ]
 [0.        ]
 [0.33333333]
 [0.66666667]
 [1.        ]]
FeatureUnion()
